In [1]:

import numpy as np
from numpy import matlib as mb
import sys
import os
import pandas as pd
import glob as glob
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import itertools
from sklearn.decomposition import PCA, IncrementalPCA
from scipy import stats
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline

In [2]:
cmap = 'husl'

In [3]:
fileDir = os.path.dirname(os.path.realpath('__file__'))

In [4]:
# figdir = '../figs'
# if not os.path.exists(figdir):
#     os.mkdir(figdir)
figdir ='../figs/replication/'
if not os.path.exists(figdir):
    os.mkdir(figdir)

In [5]:
results_dir = os.path.join(fileDir, '../../data/', 'results')

In [6]:
analysis_name = 'corrs_ordered_up_for_PCA'

In [7]:
data_dir = os.path.join(results_dir, analysis_name, 'corrs_results')

In [8]:
data_dir

'/Users/lucyowen/repos/timecorr-paper-1/code/notebooks/../../data/results/corrs_ordered_up_for_PCA/corrs_results'

In [9]:
corrsdir = os.path.join(results_dir, analysis_name)

In [10]:
corrsdir

'/Users/lucyowen/repos/timecorr-paper-1/code/notebooks/../../data/results/corrs_ordered_up_for_PCA'

In [11]:
cmap = 'husl'

In [12]:
levels = np.arange(0,4,1)
conditions = ['intact', 'paragraph', 'rest', 'word']
Reduction_type = ['PCA','eigenvector_centrality']

In [13]:
levels = np.arange(0,4,1)
conditions = ['intact', 'paragraph', 'rest', 'word']

#colors = sns.color_palette("cubehelix", 4)

for r in Reduction_type:
    param_corrs = glob.glob(os.path.join(corrsdir, 'isfc_' + r + '*'))
    for l in levels:

        full_pca_data = pd.DataFrame()

        for e, cond in enumerate(conditions):

            #con = os.path.join(param_corrs[r], f'd_{l}' + f'_r_{cond}' + '.npy')
            con = os.path.join(param_corrs[0], f'd_{l}' + f'_r_{cond}' + '.npy')
            corrs = np.load(con)

            for p in np.arange(corrs.shape[0]):

                x = corrs[p]
                pca = PCA(n_components=corrs.shape[1])
                pca.fit_transform(x)

                pc_data = pd.DataFrame()
                pc_data['variance'] = np.cumsum(pca.explained_variance_ratio_)
                pc_data['condition'] = mb.repmat(cond, np.cumsum(pca.explained_variance_ratio_).shape[0], 1)
                pc_data['subject'] = mb.repmat(p, np.cumsum(pca.explained_variance_ratio_).shape[0], 1)
                pc_data['components'] = pc_data.index 
                if full_pca_data.empty:
                    full_pca_data = pc_data
                else:
                    full_pca_data = full_pca_data.append(pc_data)
                    
    full_pca_data.to_csv(os.path.join(corrsdir, 'full_pca_data_'+ f'{r}'+'.csv'))

In [14]:
corrsdir

'/Users/lucyowen/repos/timecorr-paper-1/code/notebooks/../../data/results/corrs_ordered_up_for_PCA'

In [ ]:
levels = np.arange(0,4,1)
conditions = ['intact', 'paragraph', 'rest', 'word']

for r in Reduction_type:
    for l in levels:
        full_pca_data = pd.read_csv(os.path.join(corrsdir, 'full_pca_data_'+ f'{r}')+'.csv')
        print(str(l))            
        fig, ax = plt.subplots()
        sns.lineplot(x="components", y="variance",
                     hue="condition",
                     data=full_pca_data, ax=ax, palette=cmap, hue_order=['intact', 'paragraph', 'word', 'rest'])
        sns.despine(ax=ax, left=True)
        ax.set_ylabel('Cumulative \n variance explained',fontsize=24)
        ax.set_xlabel('Number of components',fontsize=24)
        ax.tick_params(axis='both', which='major', labelsize=24)
        ax.tick_params(axis='both', which='minor', labelsize=20)
        ax.set_ylim(0, 1)
        ax.legend().set_visible(False)

        outfile = os.path.join(figdir, 'pca_700_nodes_level_' + str(l+1) + '_' + r + '.pdf')
        plt.tight_layout()
        plt.savefig(outfile)
        plt.clf()

0
1
2
3
0
1
2
3
